# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f889f312eb0>
├── 'a' --> tensor([[-0.9196,  0.1779, -0.9184],
│                   [ 1.7881, -0.5578, -0.4313]])
└── 'x' --> <FastTreeValue 0x7f889f312af0>
    └── 'c' --> tensor([[-0.1743, -0.3635, -1.5560, -0.5748],
                        [ 1.0356,  1.4310, -0.9693, -0.7268],
                        [-0.8567, -0.1880,  1.0227,  0.2682]])

In [4]:
t.a

tensor([[-0.9196,  0.1779, -0.9184],
        [ 1.7881, -0.5578, -0.4313]])

In [5]:
%timeit t.a

87 ns ± 0.781 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f889f312eb0>
├── 'a' --> tensor([[ 0.8232,  0.0519,  0.9869],
│                   [ 1.0352, -1.0332,  0.2767]])
└── 'x' --> <FastTreeValue 0x7f889f312af0>
    └── 'c' --> tensor([[-0.1743, -0.3635, -1.5560, -0.5748],
                        [ 1.0356,  1.4310, -0.9693, -0.7268],
                        [-0.8567, -0.1880,  1.0227,  0.2682]])

In [7]:
%timeit t.a = new_value

85.6 ns ± 1.18 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9196,  0.1779, -0.9184],
               [ 1.7881, -0.5578, -0.4313]]),
    x: Batch(
           c: tensor([[-0.1743, -0.3635, -1.5560, -0.5748],
                      [ 1.0356,  1.4310, -0.9693, -0.7268],
                      [-0.8567, -0.1880,  1.0227,  0.2682]]),
       ),
)

In [10]:
b.a

tensor([[-0.9196,  0.1779, -0.9184],
        [ 1.7881, -0.5578, -0.4313]])

In [11]:
%timeit b.a

81.8 ns ± 0.632 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.6264, -0.1118, -1.1664],
               [-0.3399,  1.2319, -1.0672]]),
    x: Batch(
           c: tensor([[-0.1743, -0.3635, -1.5560, -0.5748],
                      [ 1.0356,  1.4310, -0.9693, -0.7268],
                      [-0.8567, -0.1880,  1.0227,  0.2682]]),
       ),
)

In [13]:
%timeit b.a = new_value

647 ns ± 4.14 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.03 µs ± 3.96 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.5 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

165 µs ± 2.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

167 µs ± 1.15 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f87c4b54760>
├── 'a' --> tensor([[[-0.9196,  0.1779, -0.9184],
│                    [ 1.7881, -0.5578, -0.4313]],
│           
│                   [[-0.9196,  0.1779, -0.9184],
│                    [ 1.7881, -0.5578, -0.4313]],
│           
│                   [[-0.9196,  0.1779, -0.9184],
│                    [ 1.7881, -0.5578, -0.4313]],
│           
│                   [[-0.9196,  0.1779, -0.9184],
│                    [ 1.7881, -0.5578, -0.4313]],
│           
│                   [[-0.9196,  0.1779, -0.9184],
│                    [ 1.7881, -0.5578, -0.4313]],
│           
│                   [[-0.9196,  0.1779, -0.9184],
│                    [ 1.7881, -0.5578, -0.4313]],
│           
│                   [[-0.9196,  0.1779, -0.9184],
│                    [ 1.7881, -0.5578, -0.4313]],
│           
│                   [[-0.9196,  0.1779, -0.9184],
│                    [ 1.7881, -0.5578, -0.4313]]])
└── 'x' --> <FastTreeValue 0x7f889c286190>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38.7 µs ± 704 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f87c4b54640>
├── 'a' --> tensor([[-0.9196,  0.1779, -0.9184],
│                   [ 1.7881, -0.5578, -0.4313],
│                   [-0.9196,  0.1779, -0.9184],
│                   [ 1.7881, -0.5578, -0.4313],
│                   [-0.9196,  0.1779, -0.9184],
│                   [ 1.7881, -0.5578, -0.4313],
│                   [-0.9196,  0.1779, -0.9184],
│                   [ 1.7881, -0.5578, -0.4313],
│                   [-0.9196,  0.1779, -0.9184],
│                   [ 1.7881, -0.5578, -0.4313],
│                   [-0.9196,  0.1779, -0.9184],
│                   [ 1.7881, -0.5578, -0.4313],
│                   [-0.9196,  0.1779, -0.9184],
│                   [ 1.7881, -0.5578, -0.4313],
│                   [-0.9196,  0.1779, -0.9184],
│                   [ 1.7881, -0.5578, -0.4313]])
└── 'x' --> <FastTreeValue 0x7f889c1d15b0>
    └── 'c' --> tensor([[-0.1743, -0.3635, -1.5560, -0.5748],
                        [ 1.0356,  1.4310, -0.9693, -0.7268],
                 

In [23]:
%timeit t_cat(trees)

36.2 µs ± 308 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

76.3 µs ± 383 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.9196,  0.1779, -0.9184],
                [ 1.7881, -0.5578, -0.4313]],
       
               [[-0.9196,  0.1779, -0.9184],
                [ 1.7881, -0.5578, -0.4313]],
       
               [[-0.9196,  0.1779, -0.9184],
                [ 1.7881, -0.5578, -0.4313]],
       
               [[-0.9196,  0.1779, -0.9184],
                [ 1.7881, -0.5578, -0.4313]],
       
               [[-0.9196,  0.1779, -0.9184],
                [ 1.7881, -0.5578, -0.4313]],
       
               [[-0.9196,  0.1779, -0.9184],
                [ 1.7881, -0.5578, -0.4313]],
       
               [[-0.9196,  0.1779, -0.9184],
                [ 1.7881, -0.5578, -0.4313]],
       
               [[-0.9196,  0.1779, -0.9184],
                [ 1.7881, -0.5578, -0.4313]]]),
    x: Batch(
           c: tensor([[[-0.1743, -0.3635, -1.5560, -0.5748],
                       [ 1.0356,  1.4310, -0.9693, -0.7268],
                       [-0.8567, -0.1880,  1.0227,  0.2682]],
         

In [26]:
%timeit Batch.stack(batches)

96.8 µs ± 612 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.9196,  0.1779, -0.9184],
               [ 1.7881, -0.5578, -0.4313],
               [-0.9196,  0.1779, -0.9184],
               [ 1.7881, -0.5578, -0.4313],
               [-0.9196,  0.1779, -0.9184],
               [ 1.7881, -0.5578, -0.4313],
               [-0.9196,  0.1779, -0.9184],
               [ 1.7881, -0.5578, -0.4313],
               [-0.9196,  0.1779, -0.9184],
               [ 1.7881, -0.5578, -0.4313],
               [-0.9196,  0.1779, -0.9184],
               [ 1.7881, -0.5578, -0.4313],
               [-0.9196,  0.1779, -0.9184],
               [ 1.7881, -0.5578, -0.4313],
               [-0.9196,  0.1779, -0.9184],
               [ 1.7881, -0.5578, -0.4313]]),
    x: Batch(
           c: tensor([[-0.1743, -0.3635, -1.5560, -0.5748],
                      [ 1.0356,  1.4310, -0.9693, -0.7268],
                      [-0.8567, -0.1880,  1.0227,  0.2682],
                      [-0.1743, -0.3635, -1.5560, -0.5748],
                      [ 1.0356,  

In [28]:
%timeit Batch.cat(batches)

196 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

402 µs ± 8.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
